# XGBoost 

Nesse notebook será feito:

* Análise Exploratória dos Dados
* Tratamento de Classes Desbalanceadas
* XGBoost


## Problema de Gestão

Queremos montar um modelo que seja capaz de identifcar qual casa precisa de mais assistência social da Costa Rica.

Estamos diante de um problema de **classificação multiclasse**.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# Carregando os dados
df = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')

df.shape, test.shape

In [ ]:
# Juntando os dataframes
df_all = df.append(test)

df_all.shape

In [ ]:
# Vamos visualizar os dados
df_all.info()

In [ ]:
# Vamos aumentar o número de colunas para o info mostrar
df_all.info(max_cols=145)

### Features dtype object

In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

### Dependency

De acordo com o dicionário de dados, 

> dependency, Dependency rate, calculated = (number of members of the household younger than 19 or older than 64)/(number of member of household between 19 and 64)

In [ ]:
# Olhando a coluna dependency
df_all['dependency'].value_counts()

Percebemos a grande quantidade de valores armazenados como "yes" e "no" (strings).

Visto no dicionário de dados que o 'yes' pode ser substituído por 1 e que o 'no' pode ser substituído por 0.

Vamos calcular e verificar se isso pode ser feito.

In [ ]:
#Vamos criar uma coluna com a fórmula descrita no dicionário de dados
df_all['dependency_calculated'] = (df_all['hogar_nin'] + df_all['hogar_mayor']) / (df_all['hogar_adul'])
df_all['dependency_calculated'].head()

In [ ]:
#Vamos criar uma coluna cópia de dependency, mas substituindo as strings por inteiros
df_all['dependency_test'] = df_all['dependency'].replace('yes',1).replace('no',0)
df_all['dependency_test'].head()

In [ ]:
#Vamos comparar os valores
df_all[['dependency','dependency_calculated','dependency_test','hogar_nin','hogar_mayor','hogar_adul','hogar_total']].head(20)

Analisando esses valores, percebemos que os valores que estão na coluna "dependency" não correspondem a fórmula na descrição do dicionário de dados.
A decisão nesse caso será dropar a coluna dependency que estava no dataset original (com erros de cálculo) e manter a coluna calculada.

In [ ]:
#Dropando as colunas com erro de cálculo
df_all.drop(['dependency','dependency_test'],axis=1,inplace=True)

In [ ]:
# vamos analisar se temos NA ou inf na coluna
df_all.dependency_calculated.value_counts()

In [ ]:
# setar pandas para deixar os valores inf como na
pd.set_option('mode.use_inf_as_na', True)

In [ ]:
# localizar os 36 valores infinitos
df_all.dependency_calculated.isna().sum()

In [ ]:
# substituir os valores por -1
df_all['dependency_calculated'].fillna(-1, inplace=True)
df_all.dependency_calculated.isna().sum()

### edjefa e edjefe

De acordo com o dicionário de dados,

> years of education of **female** head of household, based on the interaction of escolari (years of education), head of household and gender, yes=1 and no=0

> years of education of **male** head of household, based on the interaction of escolari (years of education), head of household and gender, yes=1 and no=0

In [ ]:
# Analisando os dados da coluna edjefa
df_all['edjefa'].value_counts()

In [ ]:
# Analisando os dados da coluna edjefe
df_all['edjefe'].value_counts()

In [ ]:
# Vamos transformar 'yes' em 1 e 'no' em 0
# nas colunas edjefa e edjefe
mapeamento = {'yes': 1, 'no': 0}

df_all['edjefa'] = df_all['edjefa'].replace(mapeamento).astype(int)
df_all['edjefe'] = df_all['edjefe'].replace(mapeamento).astype(int)

In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

In [ ]:
# Visualizando do comando info
df_all.info(max_cols=145)

### Tratamento de Valores Ausentes

In [ ]:
# Verificando os valores nulos
df_all.isnull().sum()

### Features que possuem missing values

1. v2a1 = valor aluguel mensal
2. v18q1 = quantidade de tablets que os proprietários da casa possuem
3. rez_esc = anos antes da escola
4. meaneduc = média ded anos de educação nos adultos
5. SQBmeaned = quadrado da média dos anos de educação dos adultos

In [ ]:
 # 1. Verificando os valores de aluguel (v2a1) para os chefes/as de familia (parentesco1 = 1)
df_all[df_all['parentesco1'] == 1]['v2a1'].isnull().sum()

In [ ]:
# Prenchendo com -1 os valores nulos de v2a1
df_all['v2a1'].fillna(-1, inplace=True)

A escolha para preenchimento dos valores nulos por -1 foi intencional. A ideia é não tentar excluir essa coluna e também de não perder nenhum dado. Como o algoritmo usado será Random Forest, o uso de -1 (outlier) será para "forçar" a árvore a dispensar os valores -1.

In [ ]:
# 2. Análise de v18q (relacao com v18q1)
df_all['v18q'].value_counts()

A quantidade de valores nulos na feature v18q (possui tablet), que é binária, bate com o valor de missing values da feature quantidade de tablets. Basta substituir valores nulos por 0.

In [ ]:
# Prenchendo com 0 os valores nulos de v18q1
df_all['v18q1'].fillna(0, inplace=True)

In [ ]:
# 3. Verificando valores nulos de rez_esc
print('Porcentagem de Valores Nulos:',(df_all['rez_esc'].isnull().sum() / len(df_all))*100)

In [ ]:
# Prenchendo com -1 os valores nulos de rez_esc (estratégia do outlier)
df_all['rez_esc'].fillna(-1, inplace=True)

In [ ]:
# 4. Verificando valores nulos de meaneduc
print('Porcentagem de Valores Nulos:',(df_all['meaneduc'].isnull().sum() / len(df_all))*100)
print('Quantidade de Valores Nulos:',df_all['meaneduc'].isnull().sum() )

In [ ]:
#Média e mediana de meaneduc
df_all.meaneduc.mean(), df_all.meaneduc.median()

In [ ]:
#Preenchendo os valores nulos de meaneduc com o valor de 9 anos de estudos (entre média e mediana), ou seja, uma tendência geral dos dados
df_all['meaneduc'].fillna(9, inplace=True)

In [ ]:
# 5. Verificando valores nulos de SQBmeaned
print('Porcentagem de Valores Nulos:',(df_all['SQBmeaned'].isnull().sum() / len(df_all))*100)

In [ ]:
# Prenchendo com -1 os valores nulos de SQBmeaned (estratégia do outlier)
df_all['SQBmeaned'].fillna(-1, inplace=True)


### Verificação dos dados pós tratamento

In [ ]:
#Visualização dados
df_all.info(max_cols=145)

In [ ]:
df_all.isnull().sum().sort_values()

### Visualização da Variável Target

In [ ]:
# Separando as colunas para treinamento
feats = [c for c in df_all.columns if c not in ['Id', 'idhogar', 'Target']]

In [ ]:
# Separar os dataframes
train, test = df_all[~df_all['Target'].isnull()], df_all[df_all['Target'].isnull()]

train.shape, test.shape

In [ ]:
# Histograma da Variável Target
sns.histplot(data=train, x="Target", bins = 4)
plt.show()

In [ ]:
# Verificando valores absolutos
train['Target'].value_counts()

In [ ]:
# Verificando as porcentagens
train['Target'].value_counts(normalize=True)

Estamos diante de um caso de classificação multiclasse em que as classes estão desbalanceadas (a categoria 4.0 ocupa 63% do dataset). Vamos usar **Over Sampling** para aumentar as classes minoritárias.

In [ ]:
# Dividindo dataset treino em X,y

X, y = train[feats], train[['Target']]

In [ ]:
# Importando a biblioteca
from imblearn.over_sampling import RandomOverSampler

# Fazendo o over-sampling
ros = RandomOverSampler(random_state=42)
X_ros,y_ros= ros.fit_resample(X,y)

# Verificando os dados
y_ros['Target'].value_counts()

### XGBoost após Over Sampling

In [ ]:
# Trabalhando com XGBoost
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=250, learning_rate=0.09, random_state=42)

# Treinando o modelo
xgb.fit(X_ros, y_ros)

In [ ]:
# Prever o Target de teste usando o modelo treinado
test['Target'] = xgb.predict(test[feats]).astype(int)

In [ ]:
# Vamos verificar as previsões
test['Target'].value_counts(normalize=True)

In [ ]:
# Criando o arquivo para submissão
test[['Id', 'Target']].to_csv('submission.csv', index=False)

In [ ]:
fig=plt.figure(figsize=(15, 20))

# Avaliando a importancia de cada coluna (cada variável de entrada)
pd.Series(xgb.feature_importances_, index=feats).sort_values().plot.barh()